In [1]:
import pandas as pd
import matplotlib
import math
%matplotlib inline 

In [2]:
print('DOTSCIENCE_OUTPUTS=["input_data"]')
print('DOTSCIENCE_LABELS={"model_type": "data_cleaning"}')

DOTSCIENCE_OUTPUTS=["input_data"]
DOTSCIENCE_LABELS={"model_type": "data_cleaning"}


We have taken data on the particulates in the air in London from (www.londonair.org.uk)[www.londonair.org.uk].

In particular, we chose data on:
* Nitric Oxide (NO)
* Nitrogen dioxide (N2)
* Oxides of nitrogen (NOX) 
* PM2.5 Particulate (PM2.5)
* Sulphur Dioxide (SO2)

(all measured in ug/m3).

The air was sampled at Haringey Town Hall at 15 minute intervals between 1st June 2018 and 30th June 2018.

[Query entered here](https://www.londonair.org.uk/london/asp/datasite.asp?CBXSpecies1=NOm&CBXSpecies2=NO2m&CBXSpecies3=NOXm&CBXSpecies5=PM25m&CBXSpecies6=SO2m&day1=1&month1=jun&year1=2018&day2=30&month2=jun&year2=2018&period=15min&ratidate=&site=HG1&res=6&Submit=Replot+graph)

In [3]:
df = pd.read_csv('https://www.londonair.org.uk/london/asp/downloadsite.asp?site=HG1&species1=NOm&species2=NO2m&species3=NOXm&species4=PM25m&species5=SO2m&species6=&start=1-jun-2018&end=30-jun-2018&res=6&period=15min&units=ugm3')

# Save the raw data, and point Dotscience to it.

This will put the data under version control

In [4]:
print('DOTSCIENCE_INPUTS=["gov_data_air_quality"]')

DOTSCIENCE_INPUTS=["gov_data_air_quality"]


In [5]:
df.to_csv("raw_air_data_harringey.csv")

Let's take a look at the data:

In [6]:
df

,Site,Species,ReadingDateTime,Value,Units,Provisional or Ratified
0,HG1,NO,01/06/2018 00:00,2.7,ug m-3,P
1,HG1,NO,01/06/2018 00:15,3.0,ug m-3,P
2,HG1,NO,01/06/2018 00:30,4.7,ug m-3,P
3,HG1,NO,01/06/2018 00:45,NaN,ug m-3,P
4,HG1,NO,01/06/2018 01:00,NaN,ug m-3,P
5,HG1,NO,01/06/2018 01:15,1.6,ug m-3,P
6,HG1,NO,01/06/2018 01:30,8.1,ug m-3,P
7,HG1,NO,01/06/2018 01:45,1.7,ug m-3,P
8,HG1,NO,01/06/2018 02:00,8.0,ug m-3,P
9,HG1,NO,01/06/2018 02:15,6.6,ug m-3,P


There are too many rows to view the entire dataframe. The records are sorted by `Species`. Let's see what unique values are in this column:

In [7]:
df.Species.unique()

array(['NO', 'NO2', 'NOX', 'PM2.5', 'SO2'], dtype=object)

We can see that there are five values: `NO` for nitric oxide, `NO2` for nitrogen dioxide, `NOX` for oxides of nitrogen, `Pm2.5` for PM2.5 Particulate, and `S02` for sulphur dioxide. 

How is each particulate measurement recorded? Let's look at the mesasurements for one timestamp. 

In [8]:
df.loc[df.ReadingDateTime == "01/06/2018 11:00"]

,Site,Species,ReadingDateTime,Value,Units,Provisional or Ratified
44,HG1,NO,01/06/2018 11:00,26.4,ug m-3,P
2828,HG1,NO2,01/06/2018 11:00,44.2,ug m-3,P
5612,HG1,NOX,01/06/2018 11:00,84.7,ug m-3 as NO2,P
8396,HG1,PM2.5,01/06/2018 11:00,NaN,ug m-3,P
11180,HG1,SO2,01/06/2018 11:00,NaN,ug m-3,P


It appears that each unique time has  five records: one for each of the particulates. Let's check that this is true in general.

In [9]:
datetimes_no_data =[]
for date in df.ReadingDateTime:
    if len(df.loc[df.ReadingDateTime == date]) != 5:
        datetimes_no_data.append(date)
        print(date)

if len(datetimes_no_data) == 0:
    print("no datetimes without sample data for all particulates: NO, NO2, NOX, PM2.5 and SO2!")
        

no datetimes without sample data for all particulates: NO, NO2, NOX, PM2.5 and SO2!


# Sanity check and update our datatypes

We had a quick look at our dataframe above, but let's inspect its datatypes more closely. We want to make sure that any numerical data is stored in a format that we can easily perform calulations with.

In [10]:
df.dtypes

Site                        object
Species                     object
ReadingDateTime             object
Value                      float64
Units                       object
Provisional or Ratified     object
dtype: object

`ReadingDateTime` is saved as an `object`, which in Pandas is a string. Let's save it as a proper date so that we can later on use it in date-like operations.

In [11]:
df['ReadingDateTime'] = pd.to_datetime(df['ReadingDateTime'])

In [12]:
df.dtypes

Site                               object
Species                            object
ReadingDateTime            datetime64[ns]
Value                             float64
Units                              object
Provisional or Ratified            object
dtype: object

Success! Let's save the dataframe to CSV. Dotscience will save it as a new version, so our original CSV of the same name is still accessible if we need it.

In [13]:
df.to_csv("raw_air_data_harringey.csv")

# Look for null values

Although every timestamp has a record for the five particulates we are interested in, from the part of the dataframe printed above it looks like a lot of these records are null (`NaN`). 

We want to remove null particulate `Value`s from our data. First, let's check whether any of the other columns have null values:

In [14]:
df.isnull().any()

Site                       False
Species                    False
ReadingDateTime            False
Value                       True
Units                      False
Provisional or Ratified    False
dtype: bool

OK, so we know that it is just the `Value` column that has some nulls. Let's see how many nulls it contains:

In [15]:
df['Value'].isnull().sum()

5754

Lets see how these null values are distributed

###  Null values by particulate

In [16]:
particulates = [str(array) for array in df.Species.unique()]

nulls_and_nonnulls_per_particulate = zip(particulates, 
                            [len(df.loc[(df.Value.isnull()) & (df.Species == particulate)]) for particulate in particulates],
                            [(len(df.loc[(df.Species == particulate)])) for particulate in particulates]
                           )


print("particulate: null values: non-null values:")
for (a, b, c) in nulls_and_nonnulls_per_particulate:
    print(a, "\t\t", b, "\t\t", c-b)
    
    

particulate: null values: non-null values:
NO 		 62 		 2722
NO2 		 62 		 2722
NOX 		 62 		 2722
PM2.5 		 2784 		 0
SO2 		 2784 		 0


So, all of our records for PM2.5 and SO2 are null. 62 of our records for the remaining particulates are null.

Let's go ahead and clean up the null values:


# Remove null values

All the PM2.5 and SO2 measures are null, so, first, let's remove those records altogether from our dataframe. 


The remaining particulates just have a few nulls, 62 out of 2784, or about 2%. We can probably do something useful with the remaining 98% of nonnull readings for `NO`, `NO2`, and `NOX`. Instead of treating the nulls as gaps in the data, lets replace those holes with the mean values for each respective particulate. 

### 1. Drop the PM2.5 and SO2 rows


In [17]:
rows_to_drop = df.index[df['Species'] == "SO2"].tolist() + df.index[df['Species'] == "PM2.5"].tolist()


In [18]:
len(rows_to_drop)

5568

In [19]:
df.drop(df.index[rows_to_drop], inplace = True)


In [20]:
len(df)

8352

In [21]:
df.Species.unique()

array(['NO', 'NO2', 'NOX'], dtype=object)

Looks like we sucessfully removed all the wholly-null particulates. Now, let's save this dataset. We'll give it a new name to distinguish it from the raw data saved earlier. Dotscience will keep track of the provenance of this new CSV, so we'll be able to easily see where and how it was created: namely, from raw_air_data_harringey.csv, via a transformation in this notebook. 

In [22]:
df.to_csv("nonnull_air_data_harringey.csv")

### 2. Replace sparse nulls with appropriate means
Then, we can replace those 62 null NO, NO2 and NOX measurements with their respective particulate `Value` means.

In [23]:
NO_rows = df.index[df['Species'] == "NO"].tolist()
NO2_rows = df.index[df['Species'] == "NO2"].tolist()
NOX_rows = df.index[df['Species'] == "NOX"].tolist()

NaN_NO_locs = []
NaN_NO2_locs = []
NaN_NOX_locs = []

for row in NO_rows:
    if math.isnan(df.at[row, "Value"]):
        NaN_NO_locs.append(row)
        
for row in NO2_rows:
    if math.isnan(df.at[row, "Value"]):
        NaN_NO2_locs.append(row)    
        
for row in NOX_rows:
    if math.isnan(df.at[row, "Value"]):
        NaN_NOX_locs.append(row)

In [24]:
print(len(NaN_NO_locs), len(NaN_NO2_locs), len(NaN_NOX_locs))

62 62 62


Great! We've saved the locations of the 62 `NaN` particulate values as lists. Now lets change each of these missing particulate values to the mean value for that particulate over the time sampled.

In [25]:
NO_mean = df.loc[NO_rows]["Value"].mean()
NO2_mean = df.loc[NO2_rows]["Value"].mean()
NOX_mean = df.loc[NOX_rows]["Value"].mean()

# loop through the values, updating the NaNs to the respective mean values
for nan_loc in NaN_NO_locs:
    df.at[nan_loc, 'Value'] = NO_mean
    
for nan_loc in NaN_NO2_locs:
    df.at[nan_loc, 'Value'] = NO2_mean

for nan_loc in NaN_NOX_locs:
    df.at[nan_loc, 'Value'] = NOX_mean

Now we should have removed or updated all the null values. Let's check:

In [26]:
df['Value'].isnull().sum()

0

Great! Let's celebrate by saving the null-free dataframe. We can overwrite the previous `nonnull_air_data_harringey.csv` safe in the knowledge dotsience will save the version history, so we can always roll back to the earlier version.

In [27]:
df.to_csv("nonnull_air_data_harringey.csv")

# Drop unnecessary columns

Let's take a second look at the columns of our dataframe:

In [28]:
df.head()

,Site,Species,ReadingDateTime,Value,Units,Provisional or Ratified
0,HG1,NO,2018-01-06 00:00:00,2.700000,ug m-3,P
1,HG1,NO,2018-01-06 00:15:00,3.000000,ug m-3,P
2,HG1,NO,2018-01-06 00:30:00,4.700000,ug m-3,P
3,HG1,NO,2018-01-06 00:45:00,13.953012,ug m-3,P
4,HG1,NO,2018-01-06 01:00:00,13.953012,ug m-3,P


We don't need to know the `Site` values in this case, since all our records are from the same location. So, let's drop this column.

In [29]:
df.drop(columns='Site', inplace=True, axis=1)

In [30]:
df.head()

,Species,ReadingDateTime,Value,Units,Provisional or Ratified
0,NO,2018-01-06 00:00:00,2.700000,ug m-3,P
1,NO,2018-01-06 00:15:00,3.000000,ug m-3,P
2,NO,2018-01-06 00:30:00,4.700000,ug m-3,P
3,NO,2018-01-06 00:45:00,13.953012,ug m-3,P
4,NO,2018-01-06 01:00:00,13.953012,ug m-3,P


OK, let's save this data once more:

In [31]:
df.to_csv("air_data_harringey_streamlined.csv")

In [32]:
## hacky summary stat used to try to get a commit made

In [33]:
import json

print('DOTSCIENCE_PARAMETERS=' + json.dumps({"features": "blah"}))

print('DOTSCIENCE_SUMMARY=' + json.dumps({"thing": 1}))



DOTSCIENCE_PARAMETERS={"features": "blah"}
DOTSCIENCE_SUMMARY={"thing": 1}
